<a href="https://colab.research.google.com/github/PedroTorrado/Bacon-s-Oracle-Graph/blob/main/Projeto_DAA_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Classe Vertice, Edge e Graph##

In [ ]:
# Class Vertice
class Vertex:
    ''' Estrutura de Vértice para um grafo: encapsula um elemento (vertex_id)
        que é o identificador deste nó.

        O elemento (vertex_id) deve ser hashable:
        - Um objeto hashable é aquele que pode ser utilizado como uma chave num dicionário Python.
        - Isto inclui strings, números, tuplas, etc.
    '''

    def __init__(self, vertex_id):
        '''O vértice será inserido no Grafo usando o método insert_vertex(x) que cria um Vertex'''
        self._vertex_id = vertex_id   # Id do vértice (elemento a inserir no grafo)
        self.in_time = None           # Tempo de entrada no vértice (exercício TP3)
        self.out_time = None          # Tempo de saída do vértice (exercício TP3)
        self.status = None            # Marcação de visitado/não visitado (exercício TP3)

    def __hash__(self):
        '''O valor do elemento é usado como hash para o vértice (o elemento deve ser hashable)'''
        return hash(self._vertex_id)  # devolve o hash do elemento

    def __str__(self):
        '''Devolve a representação do objeto vértice em string.'''
        if self.in_time:
            return'{0}-in({1})-out({2})'.format(self._vertex_id, self.in_time, self.out_time)
        else:
            return'{0}'.format(self._vertex_id)

    def __lt__(self, vertex):
        return self._vertex_id < vertex._vertex_id

    def __le__(self, vertex):
        return self._vertex_id <= vertex._vertex_id

    def __gt__(self, vertex):
        return self._vertex_id > vertex._vertex_id

    def __ge__(self, vertex):
        return self._vertex_id >= vertex._vertex_id

    def vertex_id(self):
        ''' Devolve o elemento guardado neste vértice.'''
        return self._vertex_id

    def isEqual(self, other_vertex):
      '''Verifica se dois vértices são iguais.'''
      return self._vertex_id == other_vertex._vertex_id


# Class Edge
class Edge:
    ''' Estrutura de Aresta para um Grafo: (origem, destino) e peso '''

    def __init__(self, vertex_1, vertex_2, name):
        self._vertex_1 = vertex_1
        self._vertex_2 = vertex_2
        self.edge_id = name

    def __hash__(self):
        # Função que mapeia a aresta a uma posição no dicionário (hash map)
        return hash( (self._vertex_1, self._vertex_2) )

    def __str__(self):
        ''' Devolve a representação do objeto aresta em string: (origem, destino)w=peso '''
        return'e({0},{1})w={2}'.format(self._vertex_1, self._vertex_2, self.edge_id)

    def __eq__(self, other):
        # define igualdade de duas arestas (deve ser consistente com a função hash)
        return self._vertex_1 == other._vertex_1 and self._vertex_2 == other._vertex_2

    def endpoints(self):
        ''' Devolve a tupla (vertex_1, vertex_2) os vértices adjacentes vertex_1 e vertex_2.'''
        return (self._vertex_1, self._vertex_2)

    def opposite(self, vertex_id):
        ''' Indica o vértice oposto ao vértice com o ID vertex_id nesta aresta
            (apenas se vertex_id fizer parte da aresta).'''
        if vertex_id == self._vertex_1.vertex_id():
            return self._vertex_2
        elif vertex_id == self._vertex_2.vertex_id():
            return self._vertex_1
        else:
            return None


    def edge_id(self):
      return self.edge_id

In [ ]:
class Graph:
    '''
    Representação de um grafo usando dicionários encadeados (nested dictionaries).

    Atributos:
    ----------
    adjancencies: Dicionário externo que associa um vértice (Vertex) a um
                  mapa de adjacências (dicionario interno)
    vertices: Dicionário auxiliar que associa o id dos vértices do grafo
              a um objeto Vertex (tabela de símbolos).
    n: Número de vértices no Grafo
    m: Número de arestas no Grafo

    ----------
'''
    def __init__(self):
        '''Construtor: Cria um grafo vazio (dicionário de _adjancencies).'''
        self._adjancencies = {}  # dicionário que associa o par chave-valor: <Vertex v, Mapa de adjacências de v>
        self._vertices = {}      # dicionário que associa o par: <id do vértice, objeto Vertex correspondente>
        self._n = 0              # número de vértices do grafo
        self._m = 0              # número de arestas do grafo

    def __str__(self):
        '''Devolve a representação do grafo em string (toString)'''
        if self._n == 0:
            ret = "DAA-Graph: <empty>\n"
        else:
            ret = "DAA-Graph:\n"
            for vertex in self._adjancencies.keys():
                #ret += "vertex-"
                ret += str(vertex) + ": "
                for edge in self.incident_edges(vertex.vertex_id()):
                    ret += str(edge) + "; "
                ret += "\n"
        return ret

    def order(self):
        '''Ordem de um grafo: a quantidade de vértices no Grafo.'''
        return self._n

    def size(self):
        '''Dimensão de um grafo: a quantidade total de arestas do Grafo.'''
        return self._m

    def has_vertex(self, vertex_id):
        '''Verifica se o vértice de id vertex_id está no grafo.'''
        return vertex_id in self._vertices

    def has_edge(self, u_id, v_id, edge_id):
        '''Verifica se a aresta (u_id, v_id) existe no grafo.'''
        if not self.has_vertex(u_id) or not self.has_vertex(v_id):
            return False
        else:
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            return vertex_v in self._adjancencies[vertex_u]

    def insert_vertex(self, vertex_id):
        '''Insere um novo vértice com o id vertex_id.'''
        if not self.has_vertex(vertex_id):
            vertex = Vertex(vertex_id)
            self._vertices[vertex_id] = vertex  # insere o novo vértice no dicionario de vertices
            self._adjancencies[vertex] = {}     # inicializa o mapa de adjacências deste vértice a vazio
            self._n +=1

    def insert_edge(self, u_id, v_id, edge_id):
        """
        Cria e insere uma nova aresta entre u_id e v_id com peso weight.
        Se a aresta já existe no grafo, atualiza-se o seu peso.
        Também insere os vértices u_id e v_id, caso não existam.
        """
        if not self.has_vertex(u_id):
            self.insert_vertex(u_id)
        if not self.has_vertex(v_id):
            self.insert_vertex(v_id)

        edge = Edge(self._vertices[u_id], self._vertices[v_id], edge_id)  # Crie um objeto Edge com edge_id

        self._adjancencies[self._vertices[u_id]][self._vertices[v_id]] = edge  # Adicione a aresta ao dicionário
        self._adjancencies[self._vertices[v_id]][self._vertices[u_id]] = edge  # Adicione a aresta ao dicionário (para arestas direcionadas)

        self._m += 1  # Atualize o número de arestas



    def incident_edges(self, vertex_id):
        '''Devolve um iterável (gerador) com todas as arestas de um vértice com id vertex_id.'''
        vertex = self._vertices[vertex_id]
        for edge in self._adjancencies[vertex].values(): # para todas as arestas incidentes em v:
            yield edge

    def degree(self, vertex_id):
        '''Quantidade de arestas incidentes no vértice v.'''
        if not self.has_vertex(vertex_id):
            return 0
        vertex = self._vertices[vertex_id]
        return len(self._adjancencies[vertex])

    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo (tipo Vertex)'''
        return self._vertices.values()

    def edges(self):
      """
      Devolve um iterável sobre todas as arestas do Grafo (sem arestas duplicadas), retornando o ID de cada aresta.
      """
      seen_edges = set()  # Set to store seen edge_id's
      for vertex in self._adjancencies:
        for edge in self._adjancencies[vertex].values():
          edge_id = edge.edge_id
          if edge_id not in seen_edges:
            seen_edges.add(edge_id)
            yield edge_id


    def remove_vertex(self, vertex_id):
        '''Remove o vértice com id vertex_id. Se o vértice não existir, não faz nada.'''
        if not self.has_vertex(vertex_id):
            return

        vertex = self._vertices[vertex_id]
        del self._vertices[vertex_id]

        for v in self._adjancencies[vertex]:
            del self._adjancencies[v][vertex]

        del self._adjancencies[vertex]

        self._n -= 1

    def remove_edge(self, u_id, v_id, edge_id):
        '''Remove a aresta entre u_id e v_id. Se a aresta não existir, não faz nada.'''
        if self.has_edge(u_id, v_id, edge_id):
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            del self._adjancencies[vertex_u][vertex_v]
            if vertex_u != vertex_v:  # laços são removidos apenas uma vez
                del self._adjancencies[vertex_v][vertex_u]
            self._m -= 1

    def get_edge_id(self, u_id, v_id):
      ''' Devolve o objeto aresta (Edge) que liga u_id a v_id.
      Devolve None se não forem adjacentes ou se (um d)os vértices não existirem.'''
      vertex_u = self._vertices[u_id]
      vertex_v = self._vertices[v_id]
      if vertex_u not in self._adjancencies or vertex_v not in self._adjancencies:
        return None
      else:
        variavel = self._adjancencies[vertex_u][vertex_v]
        print(variavel)
        return variavel

    def get_vertex_by_id(self, vertex_id):
      """
      Retrieves the Vertex object with the given ID from the graph.

      Args:
          vertex_id: The unique identifier of the vertex to find.

      Returns:
          The Vertex object with the matching ID, or None if not found.
      """
      return self._adjancencies.get(vertex_id)

    def edges_from(self, current_actor):
        """
        Returns a list of adjacent actors for the given actor.

        Args:
            current_actor: The actor for which to retrieve adjacent actors.

        Returns:
            A list of adjacent actors (vertices).
        """
        list = []
        for vertex in self._adjancencies:
          if current_actor == vertex.vertex_id(): #Se existir current_actor vai
            print("Actor found")                  #buscar os elementos adjacentes
            for edge in self._adjancencies[vertex].values(): #desse vertice
              list.append(edge)
        print(list)
        return list

    def get_edge_vertices(self, edge_id):
      """
      Returns the vertex IDs associated with the given edge ID.

      Args:
          edge_id: The ID of the edge (movie title).

      Returns:
          A list of vertexes that have edges with the given ID
      """
      vertices = set()

      for vertex in self._adjancencies:
          for edge in self._adjancencies[vertex].values():
              if edge.edge_id == edge_id:
                vertices.add(str(vertex))

      # Convert to a set to remove duplicates and then back to a list
      return list(set(vertices))

##Grupo 1.a)##

Resumo

Para o desenvolvimento deste projeto decidimos utilizar grafos não orientados, uma vez que isso é um fator que depende dos atores que queremos alcançar, consideramos tomar em consideração o número de ligações entre dois atores (por diferentes filmes) como forma de ter pesos e utilizar uma lista de adjacência pela sua reduzida ordem de complexidade em obter arestas.
De forma a encontrar-mos sempre o caminho mais curto possível e não apenas um caminho, temos também que nos certificar que utilizamos Breath-First Search (BFS) (aka Pesquisa em largura).

• O que são os vértices e as arestas no seu modelo de grafo. Qual foi o critério para esta escolha?

Por exemplo, a sua escolha facilita a implementação de alguma operação específica? Ou faz
com que as operações fiquem mais *eficientes* (em relação ao tempo e ao espaço em memória)?


Os vértices serão os atores e as arestas os filmes que ligam os mesmos. O critério baseia-se na lógica da informação, nós temos um conjunto de atores e como relação entre eles os filmes então achámos lógico que isso se traduza para o grafo como vértices e arestas respetivamente.

Acima de tudo esta implementação vai ajudar na representação final dos dados e a responder a algumas questões mais tarde que possamos fazer como em que filme participou X ator. Isto também significa que ao procurarmos a relação entre os dois atores já temos definido os filmes que também os liga, sem ser necessário procurar essa informação postriormente.

\

• A sua representação do problema resulta em que tipo de grafo (não orientado, orientado,
pesado, com multiarestas, acíclico, cíclico, bipartido, etc)?

A nossa representação resulta num grafo não orientado, com multi-arestas, conexo.


• Que tipo de modificações teve de realizar na classe Graph fornecida (teve de inserir novos
atributos/métodos e porquê?); ou como implementou a sua classe Graph?

A classe graph em comparação à classe dada na aula da semana 7, apenas foi completo com a informação que não foi fornecida no mesmo de forma a poder ser utilizado de forma correta. Para além disso a classe is_directional() foi removida uma vez que será utilizado um grafo não orientado. Foram adicionadas também as classes para Edge e Vertex e adicionados identificadores à classe Edge e removidas as funções específicas a pesos dos mesmos.


---

##Implementação dos ficheiros de dados##



In [ ]:
from google.colab import files
import os

filename = "small_dataset_utf8.txt"

# Check if the file exists
if os.path.isfile("/content/" + filename):
  print(f"File '{filename}' already exists in /content/.")
else:
  print(f"File '{filename}' not found in /content/.")
  # Upload a file
  uploaded = files.upload()

  # Check if a file was uploaded
  if uploaded:
    fileName = list(uploaded.keys())[0]  # Get the first filename

    # Read the first 10 lines of the file
    try:
      with open(fileName, 'r') as f:
        for i in range(10):
          line = f.readline()
          print(line, end='')  # Print the line without a newline
        print()  # Add a newline after printing 10 lines
    except FileNotFoundError:
      print(f"Error: File '{fileName}' not found.")

File 'small_dataset_utf8.txt' already exists in /content/.


Tivemos alguma dificuldade em ter a certeza que tinhamos sempre acesso aos ficheiros necessários quando executávamos o processo e por isso decidimos verificar diretamente na diretoria se esta se encontrava com o ficheiro necessário e se não, pedir ao utilizador para importar o mesmo.

Para isso utilizamos os recursos encontrádos em:

https://www.geeksforgeeks.org/how-to-print-all-files-within-a-directory-using-python/
https://colab.research.google.com/notebooks/io.ipynb#scrollTo=hauvGV4hV-Mh

In [ ]:
def create_movie_graph(file_name):
  """
  Creates a movie graph from a file with the following format:

  Movie Title (Year) / Actor 1 / Actor 2 / ...

  Args:
      file_name: The name of the file to read.

  Returns:
      A Graph object representing the movie relationships between actors.
  """
  movie_graph = Graph()

  try:
    with open(file_name, 'r') as f:
      for line in f:
        try:
          # Remove leading/trailing spaces and split using '/'
          split_data = line.strip().split('/')

          if len(split_data) < 2:
            raise ValueError("Unexpected line format")

          movie_title = split_data[0]
          year = split_data[1]
          actors = split_data[2:]  # All remaining elements are actors

          # Add actors as vertices (assuming unique actor names)
          for actor in actors:
            #print(actor)
            movie_graph.insert_vertex(actor)

          # Add edges between actors for the same movie (movie_title is the edge)
          for i in range(len(actors) - 1):
            for j in range(i + 1, len(actors)):
              movie_graph.insert_edge(actors[i], actors[j], movie_title)  # Use movie title as edge data
              #print(movie_title)

        except ValueError as e:
          print(f"Error: Encountered unexpected format in line: {line}")
          print(e)

  except FileNotFoundError:
    print(f"Error: File '{file_name}' not found.")

  return movie_graph

# Example usage
# movie_graph = create_movie_graph(filename)  # Replace with your file name

In [ ]:
#print('Ordem:', movie_graph.order(),'Tamanho:', movie_graph.size())
#print(movie_graph)

In [ ]:
from collections import deque

class HollywoodOracle:
    def __init__(self, filename):
        """
        Creates a HollywoodOracle object from a movie data file.

        Args:
            filename: The name of the file containing movie data.
        """
        self.movie_graph = create_movie_graph(filename)  # Replace with your file name
        self.center_of_universe = "Bacon, Kevin"
        self.actor_data = {}

    def all_movies(self):
        """
        Devolve um iterável de todos os títulos de filmes
        """
        return (movie_id for movie_id in self.movie_graph.edges())

    def all_actors(self):
      """
      Devolve um iterável de todos os atores
      """
      return (vertex.vertex_id() for vertex in self.movie_graph.vertices())

    def movies_from(self, actor):
        """
        Returns a list of unique movie titles associated with an actor.

        Args:
            actor: The actor's name (string) or ID (depending on graph implementation).
        """
        movie_titles_set = set()  # Using a set to ensure unique movie titles

        # Check if the actor is in the graph
        if self.movie_graph.has_vertex(actor):
            # For each edge incident to the actor's vertex
            for edge in self.movie_graph.incident_edges(actor):
                # Get the opposite vertex (other actor in the movie)
                other_actor = edge.opposite(actor)
                # Get the title of the movie (using the edge ID)
                movie_title = edge.edge_id
                # Add the movie title to the set
                movie_titles_set.add(movie_title)

        # Convert the set of movie titles to a list before returning
        movie_titles_list = list(movie_titles_set)
        return movie_titles_list

    def cast_of(self, movie):
      """
      Returns a list of all actors connected to the given movie.

      Args:
          movie: The movie title (string).

      Returns:
          A list of actors (vertices) connected to the given movie.
      """

      vertex = []

      # Iterate over all edges in the graph
      for edge in self.movie_graph.edges():
        # Check if the edge's ID matches the provided movie title
        if edge == movie:
          # Get the vertices associated with this edge
          vertex = self.movie_graph.get_edge_vertices(edge)
      return vertex


    def set_center_of_universe(self, actor):
      """
      Define um novo centro do universo

      Args:
          actor: nome do ator a definir como centro do universo
      """
      self.center_of_universe = actor

    def number_of_X(self, actor):
      """
      Devolve o número de bacon de dado ator

      Args:
          actor: Nome do ator
      """

      return path_to_X(actor).qsize()

    def path_to_X(self, string_actor):
      """
      devolve a sequencia de filmes e atores que o ligam ao centro do universo

      Args:
          actor: Nome do ator
      """

      # Verifica se os atores existem no grafo
      if not self.movie_graph.has_vertex(string_actor) or not self.movie_graph.has_vertex(self.center_of_universe):
            print("Nao existe esse ator no grafo")
            return None
      print("Existe o ator no grafo")


      # Inicializa a busca em largura a partir do ator inicial
      actor = [string_actor]   #transforma a string em uma lista de um elemento
      fila = deque([(actor)])  # Fila de vértices a serem visitados, cada elemento é um par (ator, caminho até agora)
      visited = set()  # Conjunto de atores já visitados


      # Realiza a busca em largura
      #Ciclo while, enquanto a fila nao esta vazia
      while fila:
          # Imprimir o conteúdo da deque antes de desempacotar
          print("Conteúdo da deque:", fila)
          current_actor, *path = fila.popleft()
          print(" Depois do poleft - Conteúdo do current actor:", current_actor)
          print("Conteúdo do path", path)
          print(" 2 - Conteúdo da fila depois do popleeft:", list(fila))


          # Verifica se encontrou o "centro do universo"
          if current_actor == self.center_of_universe:
              return path  # Retorna o caminho até o "centro do universo"


          # Marca o ator atual como visitado
          print("current_actor = ", current_actor)
          visited.add((current_actor, path[0])) #Falta corrigir o path com as arestas
          print("Conteúdo do set:", list(visited))


          # Para cada filme que o ator atual participou
          for edge in self.movie_graph.edges_from(current_actor):
              # Obtém o vértice (ator) adjacente
              adjacent_actor = edge.opposite(current_actor)
              print("O adjacent actor e: ", adjacent_actor)


              # Se o ator adjacente não foi visitado ainda
              if adjacent_actor not in visited:
                print(" 3 - Conteúdo da fila:", list(fila))
                # Adiciona o vértice adjacente à fila com o caminho até agora
                print(current_actor)
                print(adjacent_actor.vertex_id())
                add_path = [self.movie_graph.get_edge_id(current_actor, adjacent_actor.vertex_id())]
                tipo = type(add_path)
                print(tipo)
                fila.append((adjacent_actor.vertex_id(), path + add_path))


      # Se não encontrou um caminho até o "centro do universo", retorna None
      print("Nao encontrado o caminho para o centro do universo")
      return None



    def max_number_of_X(self):
      """
      Returns the maximum Bacon number among actors and the number of unconnected actors.


"""

    def count_number_of_X(self, n):
      """
      Returns the number of actors/actresses with a Bacon number equal to the given integer value (n).
      devolve o numero de atores com um numero de bacon igual a n

      Args:
          n: numero inteiro que procuramos iguais numeros de bacon
      """
      count = 0

      # Iterate through all actors in the graph
      for actor in self.all_actors():
        # If actor is not the center of the universe
          if actor != self.center_of_universe:
            # Calculate Bacon number for each actor
            bacon_number = self.number_of_X(actor)
            if bacon_number == n:
              count += 1

      return count


    def average_number_of_X(self):
      """
      devolve o número de bacon médio de todo o grafo
      """

In [ ]:
oracle = HollywoodOracle(filename)

In [ ]:

# #test all_movies
# sample_movies = list(oracle.all_movies())[:10]  # Get the first 10 movies
# print(sample_movies)

#test all_actors

sample_actors = list(oracle.all_actors())[:10]  # Get the first 10 actors

print(sample_actors)

# #test movies_from

actor = sample_actors[1]

# print(actor)
# actor_movies = oracle.movies_from(actor)
# print(actor_movies)

# #test cast_of
# movie = actor_movies[0]
# print(movie)
# movie_actors = oracle.cast_of(movie)
# print(movie_actors)

#test path_to_X
print("---------- Currently Debugging -----------")
X = oracle.path_to_X('Sorum, Ann')
print(X)

print(oravle.count_number_of_X(actor))

['Steele, Rob (I)', 'Wilson, Frank (II)', "Tingwell, Charles 'Bud'", 'Cassell, Alan (I)', 'Rodger, Ron', 'Knez, Bruno', 'Woodward, Edward', 'Cisse, Halifa', 'Quin, Don', 'Kiefel, Russell']
---------- Currently Debugging -----------
Existe o ator no grafo
Conteúdo da deque: deque([['Sorum, Ann']])
 Depois do poleft - Conteúdo do current actor: Sorum, Ann
Conteúdo do path []
 2 - Conteúdo da fila depois do popleeft: []
current_actor =  Sorum, Ann


IndexError: list index out of range